#### Ensuring environment

In [1]:
!python --version
!which python

Python 3.7.17
/home/chairs/workspace/personal-projects/lisp-compiler/quickLi/venv/bin/python


#### Exploring dis module

In [ ]:
import sys, struct
import dis 
import opcode
import types
import re

import marshal 
import struct 
import time
import imp 
import opcode 
import types

/home/chairs/workspace/personal-projects/lisp-compiler/quickLi/venv/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  # Remove the CWD from sys.path while we load stuff.


**Notes**

To compile a string containing Python source code and return a code object containing stack machine instructions and relevant memory addresses.
`compile(source, filename, mode, flag, dont_inherit, optimize)`\
`source` - String, Bytes object, or AST object. source code to compile\
`filenme` - String, location of source code string if applicable\
`mode` - robustness of call (exec, eval, single, etc.)

In [4]:
c.co_code

b'd\x00Z\x00e\x00d\x017\x00Z\x00e\x01e\x00\x83\x01\x01\x00d\x02S\x00'

The result is a bytes literal which is prefixed with b'. It is an immutable sequence of bytes and has a type of bytes. Each byte can have a decimal value of 0 to 255. So a bytes literal is an immutable sequence of integers between 0 to 255. Each byte can be shown by an ASCII character whose character code is the same as the byte value or it can be shown by a leading \x followed by two characters. The leading \x escape means that the next two characters are interpreted as hex digits for the character code.

When the interpreter executes EXTENDED_ARG, its oparg (which is 1) is left-shifted by eight bits and stored in a temporary variable. Let’s call it extended_arg (do not confuse it with the opname EXTENDED_ARG). So the binary value 0b1 (the binary value of 1) is converted to 0b100000000. This is like multiplying 1 by 256 in the decimal system and extened_arg will be equal to 256. Now we have two bytes in extened_arg. When the interpreter reaches to the next instruction, this two-byte value is added to its oparg (which is 4 here) using a bitwise or\
`EXTENDED_ARG 1`\
`CALL_FUNCTION 4`\
is interpreted as:

`EXTENDED_ARG 1`\
`CALL_FUNCTION 260`

In [2]:
# sometimes we have opargs that do not fit in the default one byte
extended_arg = 1 << 8 # same as 1 * 256
extended_arg = extended_arg | 4
# extended_arg = 256 + 4 = 260

### notes on compiler project
primitive operations in quickLi should rely on native compilation routines in the dis module rather than manually emitting the corresponding instructions into a code object. for example, 
`(+ 5 1)` should become a Python source string `5 + 1` rather than bytecode. Then we will use dis to compile it. For now. This could be accomplished using pre-fix to infix conversion with a stack.

In [22]:
"""
BST approach: covnert the prefix expression to an infix expression and use dis module to compile and run
Emit approach: use Lisp/prefix expression to parsed tokens. walk with recursive decent routine and emit an Intermediate
Representation (infix expression) and then use dis module to compile and run.
"""

''

In [ ]:

"""


postfix_expr = ''
prec_stack = []
prec_primops = {
    '+': 1,
    '-': 1,
    '*': 2,
    '/': 2
}
 http://127.0.0.1:8888/?token=6cb5749f737bfd601319e869079e0b82617f443291dce0a8en(prec_stack == 0)) and (primops_prec[tos_character] >= current_prec):
                    postfix_expr += prec_stack.pop()
                prec_stack.push(c)

    for j in range(0, len(prec_stack)):
        postfix_expr += prec_stack.pop()
"""

In [38]:


symbol_type = str 
number_type = (int, float)
boolean_type = (False, True)
list_type = list 

# primitive special forms
symbol_define = symbol_type('define')
symbol_lambda = symbol_type('lambda')
symbol_if = symbol_type('if')
symbol_set = symbol_type('set!')
symbol_quote = symbol_type('quote')
symbol_begin = symbol_type('begin')
symbol_cond = symbol_type('cond')


# primitive procedures (regular forms?)

def tokenize_source (lines, brackets=False):

    # format the 'lines' of the quickLi expression (redundant replace calls?)
    tokens = ''.join([l.rstrip() for l in lines if ';' not in l and not (l.isspace() or l == '')])\
        .replace('\t', '')\
        .replace('\n', '')\
        .replace('(', ' ( ').replace(')', ' ) ')\
        .replace('*', ' * ').replace('/', ' / ')\
        .replace('+', ' + ').replace('-', ' - ').split()

    final_tokens = [] 
    
    '''
    for tok in tokens:
        if tok in ['(', ')']:
            if brackets: final_tokens.append(tok)
            else:
                pass 
        else:
            if not bool(re.search(r'\s', tok)):
                final_tokens.append(tok)

    return final_tokens
    '''
    pattern = r'^(\(|\)|\(\(|\s)' if not brackets else r'\s'
    return [
        tok for tok in tokens if not bool(re.search(pattern, tok))
    ]

def native_type (token):
    if token == '#t': return boolean_type[1]
    if token == '#f': return boolean_type[0]
    else:
        try:
            return number_type[0](token)
        except ValueError:
            try:
                return number_type[1](token)
            except ValueError:
                return symbol_type(token)

def parse (tokens):
    # using a stack to validate balanced parenthesis and be able to go back out of recursive branches of expressions
    # (recursive s_expr descent)
    # will build a nested list holding each s_expression in desired order of execution, acting as our syntax tree
    # (since we are not storing each token as an object, we do not have the capability to stored attributes which makes debugging near impossible)
    # (this is a quick and dirty parser)
    try:
        if len(tokens) == 0: sys.exit('no tokens')
        token = tokens.pop(0)

        if token is '(':
            L = []
            while tokens[0] != ')':
                L.append(parse(tokens))
            tokens.pop(0)
            return L
        elif token is ')': sys.exit('expected \'(\' for beginning of expression')
        else:
            return native_type(token)

    except Exception:
        sys.exit('unknown parser failure')


def infix_converter (s_expr_tokens):
    stack = []

    i = len(s_expr_tokens) - 1
    while i >= 0:
        current_tok = s_expr_tokens[i]

        if not is_operator(current_tok):
            stack.append(current_tok)
            i -= 1 
        else:
            expr = "(" + stack.pop() + current_tok + stack.pop() + ")"
            stack.append(expr)
            i -= 1
    
    return stack.pop()

def is_primitive_builtin (c):
    import operator, math
    builtin_functions = {
        "*": lambda *args: reduce(operator.mul, args), 
        "+": sum, 
        "-": lambda *args: reduce(operator.sub, args), 
        "/": lambda *args: reduce(operator.div, args), 
        "^": math.pow, 
        "**": math.pow,
    }
    if c in list(builtin_functions.keys()):
        return builtin_functions[c]
    else:
        return False 



In [39]:


test_source_string = '''(/ (+ 5 (* 21323 3)) 1332222)'''
lines = test_source_string.split('\n')
tokens = tokenize_source(lines, brackets=True)      # NOTE INFIX_CONVERTER DOES NOT WORK WITH brackets=True
#ir_expr = infix_converter(tokens)
ir_expr_F = """
def F():
    print(%s)
F()
""" % ir_expr 

ir_expr_NoFunc = """ 
5 + 1
"""


# lets use the code object to get the bytecode and try to build a new code object 
# by understanding the instructions enoug. then, well try to emit the byte string
# ourself without using a pre-existing code object (i.e. a call to 'compile()')
pyvm_codeobj = compile(ir_expr_NoFunc, '', 'exec')
pyvm_codeobj_arguments = (
    pyvm_codeobj.co_argcount,
    pyvm_codeobj.co_kwonlyargcount,
    pyvm_codeobj.co_nlocals,
    pyvm_codeobj.co_stacksize,
    pyvm_codeobj.co_flags,
    pyvm_codeobj.co_code,
    pyvm_codeobj.co_consts,
    pyvm_codeobj.co_names,
    pyvm_codeobj.co_varnames,
    pyvm_codeobj.co_filename,
    pyvm_codeobj.co_name,
    pyvm_codeobj.co_firstlineno,
    pyvm_codeobj.co_lnotab, 
    pyvm_codeobj.co_freevars, 
    pyvm_codeobj.co_cellvars   
)

#print(ir_expr_F)
print(dis.dis(ir_expr_NoFunc))
exec(pyvm_codeobj)

print('--- CODE OBJECT ARGUMENTS ---')
for arg in pyvm_codeobj_arguments:
    print(str(arg))


##############################################################
# ^ EVERYTHING ABOVE IS THE CODE OBJECT RETURNED BY compile() ^ 
# BELOW WE WILL GENERATE OUR OWN


import struct 
import time 
import opcode, types 
import operator, math

code = types.CodeType 

# globals for the code object that Python will eval/exec
my_codeobj_args = []
my_codeobj = None

def fixed_compile (ir_expr_F):
    global my_codeobj, my_codeobj_args
    
    my_codeobj_args.append(0)   # first three always 0
    my_codeobj_args.append(0)  
    my_codeobj_args.append(0)
    my_codeobj_args.append(2)   # good for small programs
    my_codeobj_args.append(64)  # what does flag 64 mean?
    
""" 
taken from https://github.com/aisk/pie/
""" 
class Compiler (object):
    def __init__ (self):
        self.consts = [] 
        self.names = []
        self.varnames = []
        self.instructions = []
        self.code_length = 0

        # default return value on pyvm stack
        self.LOAD_CONST(self.make_const(None))

    def __getattr__ (self, name):
        if name not in opcode.opmap:
            return self.__getattribute__ (name)

        def opfunc (*args):
            if opcode.opname.index(name) > opcode.HAVE_ARGUMENT:
                if len(args) < 1:
                    sys.exit('not enough arguments to OPCODE')
                
                self.instructions.append(struct.pack('B', opcode.opmap[name]))
                self.instructions.append(struct.pack('H', args[0]))
                self.code_length += 3 
            else:
                self.codes.append(struct.pack('B', opcode.opmap[name]))
                self.code_length += 1
        
        return opfunc 

    def make_const (self, value):
        self.consts.append(value)
        return len(self.consts) - 1 

    def make_name(self, value):
        self.names.append(value)
        return len(self.names) - 1

    def compile (self, form):
        if type(form) == symbol_type:
            self.LOAD_FAST(0)                           # symbols (variables etc., shouldnt exist yet?)
        elif type(form) != list_type:
            self.LOAD_CONST(self.make_const(form))      # anything that is not a symbol (str) and is not a List. for now this is just numbers
        else:
            func = form[0]
            args = form[1:]

            '''in order to harvest the results of calling compile() over args list we need to pop the stack but i dont know how to do this right yet'''
            """
            if is_operator(func):                       # this just means its a builtin/primitive function which for now is just math stuff
                # usually we would want to handle builtin functions
                # i thought math operators were builtin functions by they are actually just const expressions which should be handled above?
                # later we will have true builtins which we can define in a standard library quickLi source file
                # what needs to happen is infix conversion of the current form, convert it to numeric, and recursive compile() call so it gets treated
                # as a constant by the VM and gets LOAD_CONST (0)
                operands = [self.compile(arg) for arg in args] # to ensure we have no nested operators
                infix_expression = '%s %s %s' % (operands[0], func, operands[1])
                self.compile(eval(

                ))
            """ 
            builtin = is_primitive_builtin(func)
            # this just means its a builtin/primitive function which for now is just arithmatic operators
            _lambda = lambda x: self.compile(x) if not builtin else builtin

            self.LOAD_GLOBAL(self.make_name(func))  # this means its a Python builtin function and should be processed accordingly
            map(_lambda, args)   
            self.CALL_FUNCTION(len(args))

    def build_pyvm_codeobj   


  2           0 LOAD_CONST               0 (6)
              2 RETURN_VALUE
None
--- CODE OBJECT ARGUMENTS ---
0
0
0
1
64
b'd\x00S\x00'
(None,)
()
()

<module>
1
b'\x00\x01'
()
()


In [40]:
# NOTE: fails because calling infix_expression_converter over tokens generated including parenthesis

# the 'compiler'
code_obj = compile("print("+infix_converter(tokens)+")", "", "exec")
byte_string = code_obj.co_code

ascii = [int(b) for b in byte_string]
    
# get opcode mnemonics using dis module 
i = 0
while i <= len(ascii) - 1:
    print(dis.opname[ascii[i]], ascii[i + 1])
    i += 2
# more effective
dis.dis(infix_converter(tokens))

# save the .pyc file so i can run it with python interpreter
import importlib, sys
pyc_data = importlib._bootstrap_external._code_to_timestamp_pyc(code_obj)
with open('output.pyc', 'wb') as f:
    f.write(pyc_data)

IndexError: pop from empty list

In [32]:
parse(tokens)

['/', ['+', 5, ['*', 21323, 3]], 1332222]